In [1]:
import requests
import pandas as pd
import re
import numpy as np
from IPython.core.display import HTML
from bs4 import BeautifulSoup
from datetime import datetime

In [90]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States')

In [91]:
# Create a BeautifulSoup object
soup = BeautifulSoup(response.text, 'lxml')
# use text to get the source code from the response object

In [92]:
# get title of the web page
soup.title.text

'List of presidents of the United States - Wikipedia'

In [93]:
## get all the table element with class wikitable 
table = soup.find('table', class_='wikitable')

In [94]:
## Now, lets get all the table rows for above table
table_row = table.find_all('tr')

In [95]:
## Table heading
table_row[1].get_text().strip()

'Presidency[a]\n\nPresident\n\nPrior office[b]\n\nParty[c]\n\nElection\n\nVice President'

In [96]:
column_name =(table_row[1]).get_text().strip().split('\n\n')
column_name

['Presidency[a]',
 'President',
 'Prior office[b]',
 'Party[c]',
 'Election',
 'Vice President']

In [97]:
## Remove extra character from column name
column_name = [re.sub(r'\[.*','', name) for name in column_name]
column_name

['Presidency',
 'President',
 'Prior office',
 'Party',
 'Election',
 'Vice President']

In [98]:
### Get the table data 
base_link = 'https://en.wikipedia.org/wiki/File:'
image_link = []
raw_data = []

count = 0
for i in range(2,len(table_row)):
    #print(len(table_row[i]))
    td = table_row[i].find_all('td')
    #raw_data.append(table_row[i].get_text().str.split('\n'))
    if (len(td)) >= 8:
        raw_data.append(table_row[i].get_text().strip().split('\n\n'))

        #raw_data.append(table_row[i].str.split('\n'))
        #raw_data.append(td.str.split('\n'))
        count = count+1
        whole_image_link = td[2].find('img')['src'][2:]
        image_link.append(whole_image_link)
    #print(td[i].text)  # gives period of presidency

In [99]:
# Let's see how the table data looks like
raw_data[1]

['2',
 'March 4, 1797–March 4, 1801',
 '',
 'John Adams1735–1826(Lived: 90 years)[6][7][8]',
 '1stVice president of the United States(1789–1797)',
 '',
 'Federalist',
 '1796',
 'Thomas Jefferson[g]']

In [100]:
# Convert raw data to dataframe
df = pd.DataFrame(raw_data)

In [101]:
df.shape

(45, 9)

In [102]:
df.head()

,0,1,2,3,4,5,6,7,8
0,1,"April 30, 1789[d]–March 4, 1797",,George Washington1732–1799(Lived: 67 years)[3][4][5],Commander-in-Chiefof theContinental Army(1775–1783),,Unaffiliated[2],1788–89,John Adams[e][f]
1,2,"March 4, 1797–March 4, 1801",,John Adams1735–1826(Lived: 90 years)[6][7][8],1stVice president of the United States(1789–1797),,Federalist,1796,Thomas Jefferson[g]
2,3,"March 4, 1801–March 4, 1809",,Thomas Jefferson1743–1826(Lived: 83 years)[9][10][11],2ndVice president of the United States(1797–1801),,Democratic-Republican,1800,"Aaron BurrMarch 4, 1801 – March 4, 1805"
3,4,"March 4, 1809–March 4, 1817",,James Madison1751–1836(Lived: 85 years)[12][13][14],5thUnited States secretary of state(1801–1809),,Democratic-Republican,1808,"George ClintonMarch 4, 1809 – April 20, 1812(Died in office)"
4,5,"March 4, 1817–March 4, 1825",,James Monroe1758–1831(Lived: 73 years)[15][16][17],7thUnited States secretary of state(1811–1817),,Democratic-Republican,1816,Daniel D. Tompkins


###  Preprocessing the dataframe

In [103]:
# replace field that's entirely space (or empty) with NaN
df =  df.apply(lambda x: x.str.strip()).replace('', np.nan)
# remove all the string char between square bracket as they are not needed
df = df.replace(regex=r'\[.*?\]', value='')
df.head()

,0,1,2,3,4,5,6,7,8
0,1,"April 30, 1789–March 4, 1797",NaN,George Washington1732–1799(Lived: 67 years),Commander-in-Chiefof theContinental Army(1775–1783),NaN,Unaffiliated,1788–89,John Adams
1,2,"March 4, 1797–March 4, 1801",NaN,John Adams1735–1826(Lived: 90 years),1stVice president of the United States(1789–1797),NaN,Federalist,1796,Thomas Jefferson
2,3,"March 4, 1801–March 4, 1809",NaN,Thomas Jefferson1743–1826(Lived: 83 years),2ndVice president of the United States(1797–1801),NaN,Democratic-Republican,1800,"Aaron BurrMarch 4, 1801 – March 4, 1805"
3,4,"March 4, 1809–March 4, 1817",NaN,James Madison1751–1836(Lived: 85 years),5thUnited States secretary of state(1801–1809),NaN,Democratic-Republican,1808,"George ClintonMarch 4, 1809 – April 20, 1812(Died in office)"
4,5,"March 4, 1817–March 4, 1825",NaN,James Monroe1758–1831(Lived: 73 years),7thUnited States secretary of state(1811–1817),NaN,Democratic-Republican,1816,Daniel D. Tompkins


In [104]:
# drop the NA values
df.dropna(inplace=True, axis=1,how ='all')

In [105]:
print(df.shape)
(df.head())

(45, 7)


,0,1,3,4,6,7,8
0,1,"April 30, 1789–March 4, 1797",George Washington1732–1799(Lived: 67 years),Commander-in-Chiefof theContinental Army(1775–1783),Unaffiliated,1788–89,John Adams
1,2,"March 4, 1797–March 4, 1801",John Adams1735–1826(Lived: 90 years),1stVice president of the United States(1789–1797),Federalist,1796,Thomas Jefferson
2,3,"March 4, 1801–March 4, 1809",Thomas Jefferson1743–1826(Lived: 83 years),2ndVice president of the United States(1797–1801),Democratic-Republican,1800,"Aaron BurrMarch 4, 1801 – March 4, 1805"
3,4,"March 4, 1809–March 4, 1817",James Madison1751–1836(Lived: 85 years),5thUnited States secretary of state(1801–1809),Democratic-Republican,1808,"George ClintonMarch 4, 1809 – April 20, 1812(Died in office)"
4,5,"March 4, 1817–March 4, 1825",James Monroe1758–1831(Lived: 73 years),7thUnited States secretary of state(1811–1817),Democratic-Republican,1816,Daniel D. Tompkins


In [106]:
column_name.insert(0, 'Presidency Number')
column_name

['Presidency Number',
 'Presidency',
 'President',
 'Prior office',
 'Party',
 'Election',
 'Vice President']

In [107]:
## Assign column name to dataframe
df.columns = column_name
df.tail()

,Presidency Number,Presidency,President,Prior office,Party,Election,Vice President
40,41,"January 20, 1989–January 20, 1993",George H. W. Bush1924–2018(Lived: 94 years),43rdVice president of the United States(1981–1989),Republican,1988,Dan Quayle
41,42,"January 20, 1993–January 20, 2001",Bill ClintonBorn 1946(73 years old),40th & 42ndGovernor of Arkansas(1979–1981 & 1983–1992),Democratic,1992,Al Gore
42,43,"January 20, 2001–January 20, 2009",George W. BushBorn 1946(73 years old),46thGovernor of Texas(1995–2000),Republican,2000,Dick Cheney
43,44,"January 20, 2009–January 20, 2017",Barack ObamaBorn 1961(58 years old),U.S. senator (Class 3) from Illinois(2005–2008),Democratic,2008,Joe Biden
44,45,"January 20, 2017–Incumbent",Donald TrumpBorn 1946(73 years old),President and Chairman of The Trump Organization(1971–2017)(No prior elected office),Republican,2016,Mike Pence


In [108]:
df.isnull().sum()

Presidency Number    0
Presidency           0
President            0
Prior office         0
Party                0
Election             0
Vice President       9
dtype: int64

In [109]:
## Extract only date and age from president column
df_date = df['President'].str.extractall('(\d+)').replace('\d.*', '')

## After extracting,remove all the substring except the name of presidents
df['President']=df['President'].str.replace('Born|\d.*', '')
df.tail(5)

,Presidency Number,Presidency,President,Prior office,Party,Election,Vice President
40,41,"January 20, 1989–January 20, 1993",George H. W. Bush,43rdVice president of the United States(1981–1989),Republican,1988,Dan Quayle
41,42,"January 20, 1993–January 20, 2001",Bill Clinton,40th & 42ndGovernor of Arkansas(1979–1981 & 1983–1992),Democratic,1992,Al Gore
42,43,"January 20, 2001–January 20, 2009",George W. Bush,46thGovernor of Texas(1995–2000),Republican,2000,Dick Cheney
43,44,"January 20, 2009–January 20, 2017",Barack Obama,U.S. senator (Class 3) from Illinois(2005–2008),Democratic,2008,Joe Biden
44,45,"January 20, 2017–Incumbent",Donald Trump,President and Chairman of The Trump Organization(1971–2017)(No prior elected office),Republican,2016,Mike Pence


In [110]:
df_date.tail()

0
   match      
42 1      73  
43 0      1961
   1      58  
44 0      1946
   1      73

In [111]:
df_date = df_date.unstack(level=1)
df_date.tail()

0           
match     0     1    2
40     1924  2018  94 
41     1946  73    NaN
42     1946  73    NaN
43     1961  58    NaN
44     1946  73    NaN

In [112]:
## Drop the first level column header
df_date = df_date.droplevel(level=0, axis=1)
df_date.columns.name=''  # Set columns group name to empty string
df_date.tail()

,0,1,2
40,1924,2018,94
41,1946,73,NaN
42,1946,73,NaN
43,1961,58,NaN
44,1946,73,NaN


In [120]:
df_date.columns= ['Born', 'Died', 'Age']
df_date.tail(10)

,Born,Died,Age
35,1908,1973,64
36,1913,1994,81
37,1913,2006,93
38,1924,95,NaN
39,1911,2004,93
40,1924,2018,94
41,1946,73,NaN
42,1946,73,NaN
43,1961,58,NaN
44,1946,73,NaN


In [121]:
##  Swap the value between Died and age column where there is NaN in age
df_date["Age"], df_date["Died"] = np.where(df_date['Age'].isnull(),[df_date["Died"],df_date["Age"]], [df_date["Age"], df_date["Died"] ])

In [122]:
dt.tail(10)

,Born,Died,Age
35,1908,1973,64
36,1913,1994,81
37,1913,2006,93
38,1924,NaN,95
39,1911,2004,93
40,1924,2018,94
41,1946,NaN,73
42,1946,NaN,73
43,1961,NaN,58
44,1946,NaN,73


In [123]:
## Insert above new columns in original data frame at given column index
location = [4,5,6]
df.insert(4, 'Born', df_date['Born'])
df.insert(5, 'Died', df_date['Died'])
df.insert(6, 'Age',df_date['Age'])


In [124]:
df.head()

,Presidency Number,Presidency,President,Prior office,Born,Died,Age,Party,Election,Vice President
0,1,"April 30, 1789–March 4, 1797",George Washington,Commander-in-Chiefof theContinental Army(1775–1783),1732,1799,67,Unaffiliated,1788–89,John Adams
1,2,"March 4, 1797–March 4, 1801",John Adams,1stVice president of the United States(1789–1797),1735,1826,90,Federalist,1796,Thomas Jefferson
2,3,"March 4, 1801–March 4, 1809",Thomas Jefferson,2ndVice president of the United States(1797–1801),1743,1826,83,Democratic-Republican,1800,"Aaron BurrMarch 4, 1801 – March 4, 1805"
3,4,"March 4, 1809–March 4, 1817",James Madison,5thUnited States secretary of state(1801–1809),1751,1836,85,Democratic-Republican,1808,"George ClintonMarch 4, 1809 – April 20, 1812(Died in office)"
4,5,"March 4, 1817–March 4, 1825",James Monroe,7thUnited States secretary of state(1811–1817),1758,1831,73,Democratic-Republican,1816,Daniel D. Tompkins


####  Displaying the dataframe with image of each president

In [125]:
#df.drop('image', axis=1, inplace=True)
# First insert new column in original dataframe that contain link to each image
df.insert(3, 'image',image_link)
#df.head()

In [126]:
# convert  links to html tags 
def path_to_image_html(path):
    return '<img src="'+ 'http://'+path + '" width="50" >'

pd.set_option('display.max_colwidth', -1)
HTML(df.to_html(escape=False ,formatters=dict(image=path_to_image_html)))

,Presidency Number,Presidency,President,image,Prior office,Born,Died,Age,Party,Election,Vice President
0,1,"April 30, 1789–March 4, 1797",George Washington,,Commander-in-Chiefof theContinental Army(1775–1783),1732,1799,67,Unaffiliated,1788–89,John Adams
1,2,"March 4, 1797–March 4, 1801",John Adams,,1stVice president of the United States(1789–1797),1735,1826,90,Federalist,1796,Thomas Jefferson
2,3,"March 4, 1801–March 4, 1809",Thomas Jefferson,,2ndVice president of the United States(1797–1801),1743,1826,83,Democratic-Republican,1800,"Aaron BurrMarch 4, 1801 – March 4, 1805"
3,4,"March 4, 1809–March 4, 1817",James Madison,,5thUnited States secretary of state(1801–1809),1751,1836,85,Democratic-Republican,1808,"George ClintonMarch 4, 1809 – April 20, 1812(Died in office)"
4,5,"March 4, 1817–March 4, 1825",James Monroe,,7thUnited States secretary of state(1811–1817),1758,1831,73,Democratic-Republican,1816,Daniel D. Tompkins
5,6,"March 4, 1825–March 4, 1829",John Quincy Adams,,8thUnited States secretary of state(1817–1825),1767,1848,80,Democratic-Republican,1824,John C. Calhoun
6,7,"March 4, 1829–March 4, 1837",Andrew Jackson,,U.S. senator (Class 2) from Tennessee(1797–1798 & 1823–1825),1767,1845,78,Democratic,1828,"John C. CalhounMarch 4, 1829 – December 28, 1832(Resigned from office)"
7,8,"March 4, 1837–March 4, 1841",Martin Van Buren,,8th Vice president of the United States(1833–1837),1782,1862,79,Democratic,1836,Richard M. Johnson
8,9,"March 4, 1841–April 4, 1841(Died in office)",William Henry Harrison,,United States minister to Colombia(1828–1829),1773,1841,68,Whig,1840,John Tyler(Succeeded to presidency)
9,10,"April 4, 1841–March 4, 1845",John Tyler,,10thVice president of the United States(1841),1790,1862,71,"WhigApril 4, 1841 – September 13, 1841",Office vacant,None


#### Change the Presidency column to datetime object and find the duration of presidency

In [127]:
df_time = df['Presidency'].str.split('–', expand=True)

In [128]:
df_time.head(10)

,0,1
0,"April 30, 1789","March 4, 1797"
1,"March 4, 1797","March 4, 1801"
2,"March 4, 1801","March 4, 1809"
3,"March 4, 1809","March 4, 1817"
4,"March 4, 1817","March 4, 1825"
5,"March 4, 1825","March 4, 1829"
6,"March 4, 1829","March 4, 1837"
7,"March 4, 1837","March 4, 1841"
8,"March 4, 1841","April 4, 1841(Died in office)"
9,"April 4, 1841","March 4, 1845"


In [129]:
## Change to datetime object
start_time = pd.to_datetime(df_time[0])

In [130]:
## Remove extra strings from end time column
end_time = df_time[1].str.replace(r'\(.*', '')
end_time.tail()

40    January 20, 1993
41    January 20, 2001
42    January 20, 2009
43    January 20, 2017
44    Incumbent       
Name: 1, dtype: object

In [131]:
print(pd.datetime.now().date())

2019-10-27


In [132]:
# For current president get present time
end_time.iloc[44] = pd.datetime.now().date()

In [133]:
end_time = pd.to_datetime(end_time)
end_time.tail()

40   1993-01-20
41   2001-01-20
42   2009-01-20
43   2017-01-20
44   2019-10-27
Name: 1, dtype: datetime64[ns]

In [166]:
## Get total time in office, in years
year_in_office = round((end_time - start_time)/np.timedelta64(1,'Y'),1)


In [135]:
## Now insert in original columns
df.insert(3, 'time in office (years)',year_in_office)

In [136]:
df.columns

Index(['Presidency Number', 'Presidency', 'President', 'time in offce (year)',
       'image', 'Prior office', 'Born', 'Died', 'Age', 'Party', 'Election',
       'Vice President'],
      dtype='object')

In [140]:
df[['President','time in offce (year)']].tail()

,President,time in offce (year)
40,George H. W. Bush,4.0
41,Bill Clinton,8.0
42,George W. Bush,8.0
43,Barack Obama,8.0
44,Donald Trump,2.8


### Bokeh visualization

In [141]:
## Get all the president who was in office for 8  or more years continuously
df_sorted = df[df['time in offce (year)']>=8.0]
#df_sorted = df.sort_values(by=['time in offce (year)'], axis = 0, ascending=False)

In [142]:
df_sorted.shape

(12, 12)

In [143]:
df_sorted ['image']  = 'http://'+ df_sorted ['image'] 
#df_sorted ['image'].head(2)

/home/jyoti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [144]:
from bokeh.plotting import figure, output_file, show,ColumnDataSource
from bokeh.io import output_notebook # enables plot interface in J notebook
from bokeh.models.tools import HoverTool
# init bokeh
output_notebook()

Loading BokehJS ...

In [146]:
# Create source
source = ColumnDataSource(df_sorted)
time_in_office = source.data['time in offce (year)'].tolist()
president_name = source.data['President'].tolist()
#president_name
#print(time_in_office)

In [160]:
p = figure(
  x_range=president_name,
  title = 'Presidents who were elected AND completed at least 2 consecutive full terms',
  x_axis_label ='Name',
  y_axis_label = 'Time in office (Years)',
  plot_width=600,
  plot_height=400,
  tools="pan,box_select,zoom_in,zoom_out,save,reset"
)

In [161]:
p.vbar(
    x='President', 
    top='time in offce (year)',
    fill_alpha=0.5,
    width=0.7,
    source=source
)
p.xaxis.major_label_orientation = 1

In [162]:
hover = HoverTool()
hover.tooltips = """
    <div>
        
        <div><strong>Election: </strong>@Election</div>
        <div><strong>Party: </strong>@Party</div> 
        <div><image src="@image" alt="@image" height='100' width="150"/></div>
                 
    </div>
"""


In [163]:
p.add_tools(hover)

In [164]:
show(p)